<h4>1. What is the approximate depth of a Decision Tree trained (without restrictions) on a training set with one million instances?</h4>

The depth of a well-balanced binary tree containing m leaves is equal to $log_{2}m$.

For one million instances, you then have a depth of $log_{2}(10^{6}) \approx 20$.

<h4>2. Is a node's Gini impurity generally lower or higher than it's parent's? Is it generally lower/higher, or always lower/higher? </h4>

A node's Gini impurity is generally lower than its parent's.

This is due to the CART training algorithm's cost function, which splits each node in a way that minimises the weight sum of the children's Gini impurity. However, it's possible for a child node to have one higher than its parent's, as long as this increased is more than compensated for by a decrease in the child's impurity.

<h4>3. If a Decision Tree is overfitting the training set, is it a good idea to try decreasing max_depth?</h4>

This may be a good idea, since this will constrain the model, regularising it.

<h4>4. If a Decision Tree is underfitting the training set, is it a good idea to try scaling the input features?</h4>

Scaling the input features would be a waste of time, as Decision Trees don't care about scaling.

<h4>5. If it takes one hour to train a Decision Tree on a training set containing one million instances, roughly how much time will it take to train another Decision Tree on a training set containing ten million instances? </h4>

The computational complexity of training a Decision Tree is $O(n \times m log_{2}(10m))$. So if you multiply the training set by 10, the the training time will be multiplied by $K = (n \times 10m \times log_{2}(10m)) / (n \times m log_{2}(m)) = 10 \times log_{2}(10m) / log_{2}(m)$.

So, if $m=10^{6}$, then $K \approx 11.7$, so you can expect the training time to be roughly 11.7 hours.

<h4>6. If it takes roughly one hour to train a Decision Tree on a given training set, roughly how much time will it take if you double the number of features?</h4>

If the number of features doubles, then the training time will roughly double.

<h4>7. Train and fine-tune a Decision Tree for the moons dataset by following these steps:</h4>

a. Use make_moons(n_samples = 10000, noise = 0.4) to generate a moons dataset.

In [1]:
from sklearn.datasets import make_moons

X_moons, y_moons = make_moons(n_samples = 10000, noise = 0.4, random_state = 42)

b. Use train_test_split() to split the dataset into a training set and a test set.

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_moons, y_moons,
                                                    test_size = 0.2,
                                                    random_state = 42)

c. Use grid search with cross-validation to find good hyperparameter values for a DecisionTree classifier.

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {
    'max_leaf_nodes': list(range(2, 100)),
    'max_depth': list(range(1, 7)),
    'min_samples_split': [2, 3, 4]
}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state = 42),
                              params,
                              cv = 3)

grid_search_cv.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6],
                         'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]})

In [5]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(max_depth=6, max_leaf_nodes=17, random_state=42)

d. Train it on the full training set using these hyperparameters, and measure your model's performance on the test set. 

In [6]:
from sklearn.metrics import accuracy_score

y_pred = grid_search_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.8595

<h4>8. Grow a forest by following these steps:</h4>

a. Generate 1,000 subsets of the training set, each containing 100 instances selected randomly.

In [7]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits = n_trees, test_size = len(X_train) - n_instances,
                  random_state = 42)

for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

b. Train one Decision Tree on each subset, using the best hyperparameter values found in the previous exercise. Evaluate these 1,000 Decision Trees on the test set. 

In [9]:
from sklearn.base import clone
import numpy as np

forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.8056605

c. For each test set instance, generate the prediction of the 1,000 Decision Trees, and keep only the most frequent prediction.

In [10]:
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)


/tmp/ipykernel_50855/1918334123.py:8: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)


d. Evaluate these predictions on the test set.

In [11]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.873